In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from datetime import datetime, timedelta
import pandas as pd

#current_date = pd.to_datetime(datetime.utcnow()).floor('H') - timedelta(hours=1)
current_date = pd.Timestamp('2023-01-01 09:00:00')
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2023-01-01 09:00:00')


In [10]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/27802
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2022-12-04 09:00:00 to 2023-01-01 08:00:00


2023-06-02 13:30:31,278 INFO: USE `taxi_batch_scoring_featurestore`
2023-06-02 13:30:35,403 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_batch_scoring_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '2022-12-03 09:00:00.000' AND `fg0`.`pickup_hour` <= TIMESTAMP '2023-01-02 08:00:00.000'


In [11]:
from src.inference import(
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/27802
Connected. Call `.close()` to terminate connection gracefully.

In [12]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,1.0,2023-01-01 09:00:00
1,2,0.0,2023-01-01 09:00:00
2,3,0.0,2023-01-01 09:00:00
3,4,3.0,2023-01-01 09:00:00
4,5,0.0,2023-01-01 09:00:00
...,...,...,...
260,261,18.0,2023-01-01 09:00:00
261,262,37.0,2023-01-01 09:00:00
262,263,40.0,2023-01-01 09:00:00
263,264,36.0,2023-01-01 09:00:00


Save these predictions in the feature store, so that they can be consumed by Streamlit app

In [13]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/27802
Connected. Call `.close()` to terminate connection gracefully.


In [14]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/27802/jobs/named/model_predictions_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x21fa314f6d0>, None)